In [13]:
import tensorflow as tf
import cv2
import os
import numpy as np
os.environ['TF_CPP_MIN_LEVEL'] = '2'  # to avoid absurd messages in terminal
os.environ['KMP_DUPLICATE_LIB_OK']='True'

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

from tensorflow import keras
from tensorflow.keras import layers, regularizers


model = keras.models.load_model('./emotion_detector_models/model_146.hdf5')
# emotionVector = {0: 'Neutral', 1: 'Happy', 2: 'Sad', 3:'Surprise', 4: 'Fearful', 5 : 'Disgusted', 6: 'Angry',    7: 'Contempt'}
emotionVector = {0: 'Angry', 1: 'Disgusted', 2: 'FearFul', 3:'Happy', 4: 'Neutral', 5 : 'Sad', 6: 'Surprise'}



# cap = cv2.VideoCapture('./videoplayback.mp4')
cap = cv2.VideoCapture(0)



while True:
    # Find haar cascade to draw bounding box around face
    ret, frame = cap.read()
    frame = cv2.resize(frame, (640, 360))
    if not ret:
        break
    face_detector = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # detect faces available on camera
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # take each face available on the camera and Preprocess it
    for (x, y, w, h) in num_faces:
        prediction_arg = 0
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        roi_gray_frame = cv2.resize(roi_gray_frame, (48,48))   
        roi_gray_frame = cv2.merge((roi_gray_frame,roi_gray_frame,roi_gray_frame)) 
        img = np.expand_dims(roi_gray_frame, axis = 0)
        predict_arr=model.predict(img, verbose=0)     
        
        
        prediction_arg= np.argmax(predict_arr)
#         print(prediction_arg)

        cv2.putText(frame, f'{emotionVector[prediction_arg]}', (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [23]:
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import os
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
from tensorflow.keras.utils import img_to_array

face_classifier = cv2.CascadeClassifier('./Haarcascades/haarcascade_frontalface_default.xml')
classifier = keras.models.load_model('./emotion_detector_models/model_146.hdf5')

frame = cv2.imread('./istockphoto-514382911-2048x2048.jpg')

gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)


for (x, y, w, h) in num_faces:
        prediction_arg = 0
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        roi_gray_frame = cv2.resize(roi_gray_frame, (48,48))   
        roi_gray_frame = cv2.merge((roi_gray_frame,roi_gray_frame,roi_gray_frame)) 
        img = np.expand_dims(roi_gray_frame, axis = 0)
        predict_arr=model.predict(img, verbose=0)     
        
        
        prediction_arg= np.argmax(predict_arr)
#         print(prediction_arg)

        cv2.putText(frame, f'{emotionVector[prediction_arg]}', (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    
cv2.imshow('Emotion Detection', frame)
    
if cv2.waitKey(1) & 0xFF == ord('q'):
    cap.release()
    cv2.destroyAllWindows()